# Generating UML Use Case Models from Software Requirements Using Natural Language Processing 



Method for identifying Grammatical Knowledge Pattern is modified in this version.


In [1]:
# -- coding: utf-8 --

!pip install cloudscraper
!pip install nltk

# Add spell checking library
from google.colab import drive
import sys
drive.mount('/content/drive')
# This is the path containing 'gingerit.py'.
sys.path.append('/content/drive/MyDrive/HH19')
from gingerit import GingerIt

# Add NLTK libraries for applying NLP techniques
import nltk
from nltk.stem.porter import *
from nltk import RegexpParser
from nltk.chunk import tree2conlltags

from tqdm import tqdm


# Add Stanford POS Tagger & Stanford Parser (according to [Bhatia13])
!cd ~
!wget https://nlp.stanford.edu/software/stanford-tagger-4.2.0.zip
!unzip stanford-tagger-4.2.0.zip
!wget https://nlp.stanford.edu/software/stanford-parser-4.2.0.zip
!unzip stanford-parser-4.2.0.zip

from nltk.tag import StanfordPOSTagger
tagger_jar = '/content/stanford-postagger-full-2020-11-17/stanford-postagger.jar' 
tagger_model_jar = '/content/stanford-postagger-full-2020-11-17/models/english-bidirectional-distsim.tagger'
postagger = StanfordPOSTagger(tagger_model_jar,
                path_to_jar=tagger_jar)

from nltk.parse.stanford import StanfordDependencyParser
parser_jar = '/content/stanford-parser-full-2020-11-17/stanford-parser.jar'
parser_model_jar = '/content/stanford-parser-full-2020-11-17/stanford-parser-4.2.0-models.jar'    
dependency_parser = StanfordDependencyParser(path_to_jar=parser_jar, 
                    path_to_models_jar=parser_model_jar)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 2.0 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive
--2022-10-01 10:23:07--  https://nlp.stanford.edu/software/stanford-tagger-4.2.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-tagger-4.2.0.zip [following]
--2022-10-01 10:23:07--  https://downloads.cs.stanford.edu/nlp/software/stanford-tagger-4.2.0.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP reque

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.


In [2]:
def segmentation(text):
  '''
  Dividing the paragraphs into sentences.
  -
  Empty element will be cleared if it is found in result. 
  ------------------------------------------
  Input: requirement paragraphs
  Output: a list of sentences
  '''
  result = text.split('.')
  while result[-1] == '' or result[-1] == ' ':
    result.pop()

  return result



def correction(text):
  '''
  The input sentence will be checked 
  and corrected until no mistake occurs.
  ------------------------------------------
  Input: requirement sentence
  Output: corrected sentence
  '''

  parser = GingerIt()
  while parser.parse(text)['corrections'] != []:
    text = parser.parse(text)['result']

  return text



def tokenization(sentence):
  '''
  According to [HH19], all spaces, commas and 
  full stops are replaced with spaces.
  -
  The sentence will be turned into a list of tokens.
  ------------------------------------------
  Input: sentence
  Output: a list of tokens
  '''

  # Tokenization process for segmented requirement text (Algorithm 1)
  exceptions = [',', '/', '.', ';']
  for i in range(len(sentence)):
      if sentence[i] in exceptions:
          sentence = sentence[:i] + ' ' + sentence[i+1:]
  result = sentence.lower().split()

  return result



def postag(tokens):
  '''
  Adding POS tags to the tokens.
  -
  Using Stanford POS Tagger according to [Bhatia13]
  ------------------------------------------
  Input: tokens
  Output: tokens with POS tags
  '''

  tag = postagger.tag(tokens)
  
  return tag



def chunking(pos_tag, 
      chunk_gram=r'''NP:{(<JJ.*>|<VBG>){1}<NN.*>+}
            NP:{<NN.*>{2,}}'''):
  '''
  Working along with POS tagged tokens, and rules 
  are defined using 'chunk_gram' variable.
  -
  Result of chunking is merged into a new POS
  tagged tokens.
  -
  Considering no chunking rule is mentioned in [HH19], 
  chunking can be turned off in 'get_elements' function.
  ------------------------------------------
  Input: POS tagged tokens and chunking rules
  Output: chunked and POS tagged tokens
  '''

  chunkparser = RegexpParser(chunk_gram)
  chunked   = tree2conlltags(chunkparser.parse(pos_tag))
  pos_tag_new = []
  
  for i in range(len(chunked)):
    
    word   = chunked[i][0]
    pos   = chunked[i][1]
    chunk  = chunked[i][2]

    if chunk == 'O':
      if i!=0:
        if chunked[i-1][2] != 'O':
          pos_tag_new.append((w_new, chunked[i-1][2].split('-')[1]))
      pos_tag_new.append((word,pos))

    elif 'B-' in chunk:
      w_new = word
    
    else:
      w_new += ('|'+word)
      pos = chunk.split('-')[1]
      if i==len(chunked)-1:
        pos_tag_new.append((w_new,pos))

  return pos_tag_new



def stemming(word):
  '''
  Remove changes of a word, turning it back to its root.
  -
  Porter’s stemming algorithm is used according to [HH19].
  ------------------------------------------
  Input: word/phrase
  Output: stemmed word/phrase
  '''

  stemmer = PorterStemmer()
  word = word.split()
  word = ' '.join([stemmer.stem(w) for w in word])

  return stemmer.stem(word)

In [3]:
def GKP(chunked_pos):
  '''
  A modified method based on [Bhatia13], which is 
  able to process sentences containing clauses.
  Using dependency parsing from Stanford Parser to 
  identify Grammatical Knowledge Patterns.
  -
  The result is stored in a dictionary, containing 
  GKP pattern(s), Subject/doer, Verb and Object.
  ------------------------------------------
  Input: POS tagged tokens
  Output: a dict containing GKP pattern(s) and GKP elements
  '''
  dependency = dependency_parser.tagged_parse(chunked_pos)
  dependency_list = list(dependency)
  root_deps = dependency_list[0].root['deps']

  GKP_tags = []

  obj_address    = root_deps['obj']
  nsubj_address   = root_deps['nsubj']   # Exist if the subject is a noun
  obl_address    = root_deps['obl']
  nsubjpass_address = root_deps['nsubj:pass']
  xcomp_address   = root_deps['xcomp']
  discourse_address = root_deps['discourse'] # Exist if the subject is a noun phrase

  verb = dependency_list[0].root
  noun_phrase_exist = discourse_address!=[]
  subject_exist = ((nsubj_address!=[]) or noun_phrase_exist or (obl_address!=[]))

  # check whether it contains open clausal complement
  if xcomp_address != [] and subject_exist:
    xcomp = dependency_list[0].get_by_address(xcomp_address[0])
    if xcomp['deps']['obj'] != []:
      temp_dic = {'type': 'complement_clause'}

      if obl_address!=[]:
        subject = dependency_list[0].get_by_address(obl_address[0])
      else:
        subject = dependency_list[0].get_by_address(discourse_address[0]) if noun_phrase_exist else dependency_list[0].get_by_address(nsubj_address[0])
      obj   = dependency_list[0].get_by_address(xcomp['deps']['obj'][0])

      temp_dic['subject'] = subject['word']
      temp_dic['obj']   = obj['word']
      temp_dic['verb']  = xcomp['word']

      GKP_tags.append(temp_dic)

  # check whether it is 'active_voice pattern'
  if obj_address != [] and subject_exist:
    temp_dic = {'type': 'active_voice'}

    if obl_address!=[]:
      subject = dependency_list[0].get_by_address(obl_address[0])
    else:
      subject = dependency_list[0].get_by_address(discourse_address[0]) if noun_phrase_exist else dependency_list[0].get_by_address(nsubj_address[0])
    obj   = dependency_list[0].get_by_address(obj_address[0])

    temp_dic['subject'] = subject['word']
    temp_dic['obj']   = obj['word']
    temp_dic['verb']  = verb['word']

    GKP_tags.append(temp_dic)

  # check whether it is 'passive_voice pattern'
  if nsubjpass_address != []:
    temp_dic = {'type': 'passive_voice'}    
    obj = dependency_list[0].get_by_address(nsubjpass_address[0])

    temp_dic['subject'] = []
    # check whether the subject is contained
    if obl_address != []:
      subject = dependency_list[0].get_by_address(obl_address[0])
      temp_dic['subject'] = subject['word']

    temp_dic['verb']  = verb['word']
    temp_dic['obj']   = obj['word']
    
    GKP_tags.append(temp_dic)

  return GKP_tags



def get_actor(pos_tag, gkp_tag,
              stem,
              sys_desc,
              my_sys_name,
              verb_types=['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'],
              noun_types=['NN', 'NNS', 'NNP', 'NNPS', 'NP']):
  '''
  Using R1, R2 and Algorithm 2 mentioned in [HH19] to
  determine the actor of a sentence.
  -
  Description name of the system can be modified in 'sys_desc',
  and the name of system should be defined by 'my_sys_name'.
  -
  'verb_types' contains POS tags considering all types of verbs,
  and 'noun_types' is the list of POS tags including all types of nouns. 
  ------------------------------------------
  Input: POS tagged tokens, GKP of the token
  Output: a str of actor (if actor does not exist, return None)
  '''

  actor = None

  if gkp_tag['subject'] != []: # R1
    subject = ' '.join(gkp_tag['subject'].split('|'))
    actor = stemming(subject) if stem else subject

    # Identify external systems as actors (Algorithm 2)
    if ((subject.split()[-1] in sys_desc) and 
      (subject.split()[:-1] == my_sys_name.lower())):
      return None

  else:
    # R2               
    for i in range(len(pos_tag)):
      if pos_tag[i][1] == 'PRP':
        if i == 0:
          actor = stemming(pos_tag[i][0]) if stem else pos_tag[i][0]
          break
        elif pos_tag[i-1][1] not in verb_types:
          actor = stemming(pos_tag[i][0]) if stem else pos_tag[i][0]
          break

  return actor



def get_usecase(tokens, pos_tag, gkp_tag, stem):
  '''
  Using R3, R4 and Algorithm 3 mentioned in [HH19] to
  determine the usecase of a series of tokens.
  ------------------------------------------
  Input: Tokens, POS tagged tokens, GKP of the token
  Output: a str of actor (if actor does not exist, return None)
  '''

  usecase = set()
  # R3
  verb = gkp_tag['verb']
  obj = ' '.join(gkp_tag['obj'].split('|'))
  verbs = [verb]
  # R4 has been inplemented because helping verbs
  # are excluded by dependency parsing in GKP.

  # Identify exceptional use cases (Algorithm 3)
  verb_id = tokens.index(verb)
  if verb_id != 0:
    if pos_tag[verb_id][1] == pos_tag[verb_id-1][1]:
      verbs.append(tokens[verb_id-1])
  if verb_id != len(tokens)-1:
    if pos_tag[verb_id][1] == pos_tag[verb_id+1][1]:
      verbs.append(tokens[verb_id+1])
  
  for v in verbs:
    if stem:
      usecase.add(stemming(v)+' '+stemming(obj))
    else:
      usecase.add(v+' '+obj)
  return list(usecase)



def get_elements(tokens,
            chunk,
            chunk_grams,
            stem, 
            sys_desc, 
            my_sys_name):
  '''
  Tokens will be POS tagged and chunked (or not), then actor(s) 
  and usecase(s) of a sentence will be found and stored in a 
  set as pair(s) by 'get_actor' & 'get_usecase' functions.
  -
  Multiple (actor, usecase) pairs may be found because 
  different GKP categories can exist in one sentence.
  ------------------------------------------
  Input: tokens
  Output: a set of (actor, usecase) pair(s)
  '''

  pos_tag = postag(tokens)
  
  if chunk:
    if chunk_grams == []:
      pos_tag = chunking(pos_tag)
    else:
      for i in range(len(chunk_grams)):
        pos_tag = chunking(pos_tag, chunk_gram=chunk_grams[i])

  GKP_tags = GKP(pos_tag)
  # Rearranging tokens accoring to the (chunking) result.
  tokens = [pos_tag[i][0] for i in range(len(pos_tag))]

  elements = set()
  if GKP_tags == []:
    return 'No GKP found.' # No Grammatical Knowledge Pattern found. 

  for gkp_tag in GKP_tags:
    actor = get_actor(pos_tag, gkp_tag, stem, sys_desc, my_sys_name)
    if actor is not None:
      usecases = get_usecase(tokens, pos_tag, gkp_tag, stem)
      if usecases != []:
        for usecase in usecases:
          elements.add((actor, usecase))
  return elements



def hh19(document_name,
         chunk=True, 
         chunk_grams=[],
         stem=True,
         sys_desc=['system', 'application'],
         my_sys_name=''):
  '''
  The main function of this program.
  -
  Input document(.txt) will be opened. Then the text inside document
  will be segmented into a list of sentences. After corrections,
  each sentence will be passed into the 'get_elements' function to 
  get its actor(s) and usecase(s).
  -
  Chunking processing during depedency parsing can be turned off by 
  setting the variable 'chunk' to False. And a list of rules for 
  chunking can be modified in 'chunk_grams' variable.
  -
  Stemming processing for the output can be turned off by setting 
  the variable 'stem' to False.
  -
  Description name of the system can be modified in 'sys_desc',
  and the name of system should be defined by 'my_sys_name'.
  -
  The result would be merged and stored as a list of pairs.
  ------------------------------------------
  Input: name of the input document
  Output: a list of (actor, usecase) pair(s)
  '''
  
  with open('/content/drive/MyDrive/HH19/inputs/'+document_name) as document:
    document = document.read()
  document = document.replace('\n', '')

  sents = segmentation(document)
  elements = []

  for sent in tqdm(sents):
    sent = correction(sent)
    elements.append(get_elements(tokenization(sent), chunk, chunk_grams, stem, sys_desc, my_sys_name))

  return elements

In [4]:
res_k3ucs = hh19('k3ucs.txt')
res_fabsucs = hh19('FABSucs.txt')

100%|██████████| 98/98 [15:15<00:00,  9.34s/it]


In [5]:
res_k3ucs

[{('program administr', 'add clinic class')},
 {('program administr', 'add new cohort')},
 {('program administr', 'add new class')},
 {('nurs staff memb', 'creat new program')},
 {('program administr', 'have abil')},
 {('system', 'display report')},
 {('program administr', 'add clinic sit')},
 {('program administr', 'add clinic lab sect')},
 {('program administr', 'add student')},
 {('system', 'allow program administr')},
 {('system', 'allow program administr nurs staff memb'),
  ('system', 'move student')},
 {('nurs staff memb', 'cancel clinic lab sect')},
 {('program administr nurs staff memb', 'add new nurs stud')},
 {('staff memb', 'have abil')},
 {('system', 'display printabl summari')},
 {('system', 'display printabl summari')},
 {('system', 'display printabl summari')},
 {('system', 'display printabl summari')},
 {('system', 'display printabl summari')},
 {('system', 'notifi affect parti')},
 {('system', 'notifi affect parti')},
 {('nurs staff memb', 'have abil')},
 {('nurs staf

In [6]:
res_fabsucs

[{('ui design', 'redesign resourc pag')},
 'No GKP found.',
 'No GKP found.',
 'No GKP found.',
 'No GKP found.',
 'No GKP found.',
 {('develop', 'add updat')},
 {('devop engin', 'provid use data'), ('devop engin', 'want new rel')},
 'No GKP found.',
 'No GKP found.',
 {('broker us', 'upload error messag')},
 'No GKP found.',
 {('websit us', 'want fab fil')},
 'No GKP found.',
 {('develop', 'updat broker valid rule tabl')},
 'No GKP found.',
 {('develop', 'want d file gener request')},
 {('user', 'access file')},
 'No GKP found.',
 'No GKP found.',
 {('ui design', 'track issu')},
 'No GKP found.',
 {('ui design', 'begin user test')},
 'No GKP found.',
 {('owner', 'design schedul')},
 {('owner', 'design audit')},
 {('develop', 'prevent user')},
 {('data us', 'receiv updat')},
 'No GKP found.',
 {('develop', 'updat fab sampl fil')},
 'No GKP found.',
 {('websit us', 'see financi assist data')},
 {('user', 'want publish')},
 'No GKP found.',
 {('owner', 'reset environ')},
 {('user', 'want